In [1]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import r as R
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri
import numpy as np
import sqlite3 as sql
import pandas.io.sql as pdsql
import re
import struct
import binascii

In [2]:
db = "/ifs/projects/proj052/pipeline_proj052/csvdb"
dbh = sql.connect(db)
pandas2ri.activate()
numpy2ri.activate()

# have to pull out arrays in R
# need to unserialize in R as well, Python doesn't reconstruct the arrays properly
R('''library(RSQLite)''')
R('''con <- dbConnect(SQLite(), "%s")''' % db)
R('''out_df <- dbGetQuery(con, "SELECT * from CD4_Tmem_fano")''')
R('''out_df$rearrays <- lapply(out_df$array, FUN=function(x) {unserialize(charToRaw(x))})''')

# drop the raw arrays
py_df = R["out_df"]

# arrays are in column 5
gates = [gx for gx in py_df[0]]
batch = [bx for bx in py_df[1]]
rows = [rx for rx in py_df[2]]
cols = [cx for cx in py_df[3]]
raw_arrays = [rax for rax in py_df[4]]
arrays = [ax for ax in py_df[5]]
indx = range(len(arrays))

/ifs/apps/apps/python-2.7.10/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: DBI

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [3]:
df = pd.DataFrame(columns = ["gates", "batch", "twin.ids", "params", "arrays"],
                  index=indx)
df["arrays"] = arrays
df["gates"] = gates
df["batch"] = batch
df["twin.ids"] = rows
df["params"] = cols
df.head()

,gates,batch,twin.ids,params,arrays
0,FSC-A.FSC-H.SSC-A,ZZEV,Twin_TS01_1/Twin_TS01_10/Twin_TS01_11/Twin_TS0...,CD161/NA/PD-1/Aq/NA/CD8/CCR6/CD45RA/NA/CD4/CXC...,"[[0.0, 0.000382912816514, 0.000259347407308, 0..."
1,FSC-A.FSC-H.CD161,ZZEV,Twin_TS01_1/Twin_TS01_10/Twin_TS01_11/Twin_TS0...,CD161/NA/PD-1/Aq/NA/CD8/CCR6/CD45RA/NA/CD4/CXC...,"[[0.0, 0.000394743593717, 0.0, 0.0, 0.00013117..."
2,FSC-A.FSC-H.PD-1,ZZEV,Twin_TS01_1/Twin_TS01_10/Twin_TS01_11/Twin_TS0...,CD161/NA/PD-1/Aq/NA/CD8/CCR6/CD45RA/NA/CD4/CXC...,"[[0.0, 0.000120365756097, 1.78624042582e-05, 0..."
3,FSC-A.FSC-H.Aq,ZZEV,Twin_TS01_1/Twin_TS01_10/Twin_TS01_11/Twin_TS0...,CD161/NA/PD-1/Aq/NA/CD8/CCR6/CD45RA/NA/CD4/CXC...,"[[0.0, 0.000388163279044, 0.000360529731116, 0..."
4,FSC-A.FSC-H.CD8,ZZEV,Twin_TS01_1/Twin_TS01_10/Twin_TS01_11/Twin_TS0...,CD161/NA/PD-1/Aq/NA/CD8/CCR6/CD45RA/NA/CD4/CXC...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [10]:
rownames = len(df["twin.ids"][505].split("/"))
colnames = len(df["params"][505].split("/"))
gate = df["gates"][505].split(".")
array = np.array(df["arrays"][505]).reshape(rownames, colnames)
print len(df["gates"])
print pd.DataFrame(array, columns=df["params"][505].split("/"),
                  index=df["twin.ids"][505].split("/"))
#pd.DataFrame(np.fromstring(str(df["array"][125]), dtype=np.float64, count=rows*cols).reshape((rows, cols)),
#             columns=df["columns"][125].split("/"), index=df["rows"][125].split("/"))

560
['CD45RA', 'CD4', 'CCR7']
                  CD161         NA       PD-1         Aq         NA  \
Twin_TS01_1    0.000000   0.000383   0.000259   0.000000   0.000096   
Twin_TS01_10   0.000000   0.000000   0.000000   0.000000   0.000107   
Twin_TS01_11   0.000000   0.000000   0.000434   0.000000   0.000324   
Twin_TS01_12   0.073547   0.000000   0.002435   0.053324   0.000000   
Twin_TS01_13   0.000000   0.000000   0.117630   0.000000   0.126612   
Twin_TS01_14  25.492242   0.000000  41.644086   0.000000  38.305760   
Twin_TS01_15   3.275560  24.944575   0.000000   0.000000   0.000000   
Twin_TS01_16  33.688433   0.000000  10.044140   0.000000   0.000000   
Twin_TS01_17   5.626367   0.000000  27.772179   0.000000   6.811693   
Twin_TS01_18   0.000000   0.000000   0.000000   0.000000   0.000000   
Twin_TS01_19   0.701811   0.000000   0.000000  34.505209   0.000000   
Twin_TS01_2   22.663834   0.000000  37.553462  33.622102   0.000000   
Twin_TS01_20   0.000000   0.000000   0.000000  

In [ ]:
rows = len(df["rows"][505].split("/"))
cols = len(df["columns"][505].split("/"))
pd.DataFrame(np.fromstring(str(df["array"][505]), dtype=np.float16, count=rows*(cols-1)).reshape((rows, (cols-1))),
             columns=df["columns"][505].split("/")[:-1], index=df["rows"][505].split("/"))

In [ ]:
[x for x in df["gate"] if re.search("CD8.CD4", x)]

In [ ]:
R('''library(RSQLite)''')
R('''con <- dbConnect(SQLite(), ":memory:")''')
R('''dbGetQuery(con, "CREATE TABLE IF NOT EXISTS test_table (array BLOB)")''')
R('''array1 <- matrix(c(0, 1.34, 8.23, 23.23, 1.35, 0.2342, 0.4, 0.12314, 0.2141242, '''
'''4.324, 0.132512, 0.1235315, 0.1225, 0.234, 0.45642, 0.45724, 0.2745, 0.12351), ncol=4, nrow=4)''')
R('''array2 <- matrix(seq(1:16), ncol=4, nrow=4)''')

R('''list_of_arrays = list(array1, array2)''')
R('''df <- data.frame(indx=1:length(list_of_arrays),
    arrays=I(unlist(lapply(list_of_arrays, FUN=function(x) {rawToChar(serialize(x, NULL, ascii=T))}))))''')

R('''dbGetPreparedQuery(con, "INSERT into test_table (array) values (:arrays)", bind.data=df)''')
R('''rearray1 <- unserialize(charToRaw(raw_char1))''')
R('''out_arrays <- df$arrays''')
R('''rearrays <- lapply(df$arrays, FUN=function(x) {unserialize(charToRaw(x))})''')

R('''print(!is.finite(Inf))''')
pandas2ri.activate()
py_arrays = R["rearrays"]
py_arrays[1]

In [ ]:
#print np.frombuffer(df["array"][505], count=rows*cols, dtype=np.float16).reshape(rows, cols)